# ACO Hyperparameter Tuning for Graph Coloring Problem

This notebook performs hyperparameter tuning for Ant Colony Optimization (ACO) algorithm applied to the Graph Coloring Problem using Optuna.

## Workflow
1. **Environment Setup**: Detect execution environment (Colab/Local) and configure paths
2. **Parameter Configuration**: Define hyperparameter search space
3. **Optimization**: Run Optuna tuner with objective function
4. **Testing**: Evaluate best parameters on testing dataset
5. **Visualization**: Generate and save performance plots

## 1. Environment Detection and Setup

In [ ]:
import sys
import os

# Check if running in Google Colab environment
IS_COLAB = 'google.colab' in sys.modules

print(f"Running in Google Colab: {IS_COLAB}")

if IS_COLAB:
    print("Colab environment detected. Will mount Google Drive.")
    # Mount Google Drive if running in Colab
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully at /content/drive")
else:
    print("Local environment detected. Using local paths.")

Running in Google Colab: False
Local environment detected. Using local paths.


## 2. Path Configuration

Configure paths for data, studies, results, and figures based on the execution environment.

In [3]:
from pathlib import Path

# Configure base paths based on environment
if IS_COLAB:
    # Update this path to match your Google Drive structure
    BASE_PATH = Path('/content/drive/MyDrive/meta_graph_coloring_antcol/assignemnt3')
    CODE_PATH = BASE_PATH / 'code'
    # Add code path to system path for imports
    sys.path.insert(0, str(CODE_PATH))
else:
    # Local environment paths
    BASE_PATH = Path('/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3')
    CODE_PATH = BASE_PATH / 'code'

# Define data root path (contains tiny_dataset and main_dataset)
DATA_ROOT = BASE_PATH / 'data'

# Verify paths exist
if not BASE_PATH.exists():
    raise FileNotFoundError(f"Base path does not exist: {BASE_PATH}")
if not DATA_ROOT.exists():
    raise FileNotFoundError(f"Data root does not exist: {DATA_ROOT}")

print(f"Base Path: {BASE_PATH}")
print(f"Code Path: {CODE_PATH}")
print(f"Data Root: {DATA_ROOT}")
print(f"\nPath verification: OK")

Base Path: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3
Code Path: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/code
Data Root: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data

Path verification: OK


## 3. Install Dependencies (Colab Only)

In [4]:
# Install required packages if running in Colab
if IS_COLAB:
    print("Installing required packages...")
    !pip install -q networkx==3.2.1 matplotlib==3.8.2 pandas==2.1.4 numpy==1.26.2 optuna==3.5.0
    print("Packages installed successfully!")

## 4. Import Required Libraries

In [5]:
import multiprocessing
from datetime import datetime
from IPython.display import Image, display

# Import project modules
from dataloader import GraphDataLoader
from optuna_tuner import OptunaACOTuner
from aco_gpc import ACOGraphColoring
from objective_function import aco_objective_function
from results_utils import visualize_testing_results, export_results, print_summary_statistics, print_file_locations

print("All libraries imported successfully!")

/Users/mahdy/projects/meta_graph_coloring_antcol/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All libraries imported successfully!


## 5. Configuration

Define dataset selection, study name, and hyperparameter search space.

In [6]:
# Dataset selection: 'tiny_dataset' for quick testing, 'main_dataset' for full experiments
DATASET_NAME = 'tiny_dataset'  # Change to 'main_dataset' for full tuning

# Study name (will be used for study files and result organization)
STUDY_NAME = f'aco_study_{DATASET_NAME}_{datetime.now().strftime("%Y%m%d_%H%M%S")}'

# Number of Optuna trials for hyperparameter tuning
N_TRIALS = 5

# ACO verbose setting 
ACO_VERBOSE = True   # Set to True to see detailed ACO progress

# Hyperparameter search space configuration (parameters to optimize)
PARAM_CONFIG = {
    'iterations': {
        'type': 'int',
        'low': 50,
        'high': 200,
    },
    'num_colors': {
        'type': 'int',
        'low': 20,
        'high': 50,
    },
    'alpha': {
        'type': 'float',
        'low': 0.5,
        'high': 3.0,
    },
    'beta': {
        'type': 'float',
        'low': 1.0,
        'high': 5.0,
    },
    'rho': {
        'type': 'float',
        'low': 0.01,
        'high': 0.5,
    },
    'ant_count': {
        'type': 'int',
        'low': 10,
        'high': 50,
    },
    'Q': {
        'type': 'float',
        'low': 0.1,
        'high': 10.0,
    }
}

print(f"Configuration:")
print(f"  Dataset: {DATASET_NAME}")
print(f"  Study Name: {STUDY_NAME}")
print(f"  Optuna Trials: {N_TRIALS}")
print(f"  Note: ACO uses parallel threads internally")
print(f"\nHyperparameters to Optimize:")
for param_name, param_spec in PARAM_CONFIG.items():
    print(f"  {param_name}: [{param_spec['low']}, {param_spec['high']}] ({param_spec['type']})")

Configuration:
  Dataset: tiny_dataset
  Study Name: aco_study_tiny_dataset_20251129_162417
  Optuna Trials: 5
  Note: ACO uses parallel threads internally

Hyperparameters to Optimize:
  iterations: [50, 200] (int)
  num_colors: [20, 50] (int)
  alpha: [0.5, 3.0] (float)
  beta: [1.0, 5.0] (float)
  rho: [0.01, 0.5] (float)
  ant_count: [10, 50] (int)
  Q: [0.1, 10.0] (float)


## 6. Run Hyperparameter Optimization

Run Optuna optimization using the objective function from separate script.

In [7]:
# Wrapper function to pass additional parameters to objective function
def objective_wrapper(trial, params, data_loader, aco_class, fixed_params):
    return aco_objective_function(
        trial=trial,
        params=params,
        data_loader=data_loader,
        aco_class=aco_class,
        verbose=ACO_VERBOSE
    )

print("Objective function wrapper ready!")

Objective function wrapper ready!


## 7. Initialize Tuner and Run Optimization

In [8]:
# Initialize the Optuna tuner
tuner = OptunaACOTuner(
    study_name=STUDY_NAME,
    data_root=str(DATA_ROOT),
    dataset_name=DATASET_NAME,
    direction='minimize'  # We want to minimize the number of colors used
)

print(f"Tuner initialized with study: {STUDY_NAME}")
print(f"Data root: {DATA_ROOT}")
print(f"Dataset: {DATASET_NAME}")

Tuner initialized with study: aco_study_tiny_dataset_20251129_162417
Data root: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data
Dataset: tiny_dataset


/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/code/optuna_tuner.py:64: FutureWarning: JournalFileStorage has been deprecated in v4.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v4.0.0. Use :class:`~optuna.storages.journal.JournalFileBackend` instead.
  self.storage = JournalStorage(JournalFileStorage(str(self.journal_file)))


In [9]:
# Run the hyperparameter optimization
print(f"\nStarting hyperparameter optimization with {N_TRIALS} trials...")
print("=" * 70)

best_params = tuner.optimize(
    objective_func=objective_wrapper,
    param_config=PARAM_CONFIG,
    aco_class=ACOGraphColoring,
    n_trials=N_TRIALS
)

print("\n" + "=" * 70)
print("Optimization completed!")
print("\nBest Parameters Found:")
for param_name, param_value in best_params.items():
    print(f"  {param_name}: {param_value}")

[I 2025-11-29 16:24:17,399] A new study created in Journal with name: aco_study_tiny_dataset_20251129_162417



Starting hyperparameter optimization with 5 trials...
Created new study 'aco_study_tiny_dataset_20251129_162417'
Completed trials: 0/5
Remaining trials: 5

Starting optimization with 5 trials...


  0%|          | 0/5 [00:00<?, ?it/s]


Loading Tuning Dataset: tiny_dataset
  gc_20_9:
    Nodes: 22, Edges: 165
    Density: 0.7143, Connected: False
Iter 1: iter_best colors=17, conflicts=0 | global_best colors=17, conflicts=0
Perfect coloring (0 conflicts) found — stopping early.
  [gc_20_9] Colors: 17, Conflicts: 0
  gc_4_1:
    Nodes: 5, Edges: 4
    Density: 0.4000, Connected: True
Iter 1: iter_best colors=4, conflicts=0 | global_best colors=4, conflicts=0
Perfect coloring (0 conflicts) found — stopping early.
  [gc_4_1] Colors: 4, Conflicts: 0
  gc_50_9:
    Nodes: 52, Edges: 1104
    Density: 0.8326, Connected: False
Iter 1: iter_best colors=33, conflicts=7 | global_best colors=33, conflicts=7
Iter 2: iter_best colors=33, conflicts=7 | global_best colors=33, conflicts=7
Iter 2: iter_best colors=33, conflicts=7 | global_best colors=33, conflicts=7
Iter 3: iter_best colors=33, conflicts=5 | global_best colors=33, conflicts=5
Iter 4: iter_best colors=33, conflicts=7 | global_best colors=33, conflicts=5
Iter 3: iter_be

Best trial: 0. Best value: 4:  20%|██        | 1/5 [00:07<00:31,  7.78s/it]

...

Best trial: 0. Best value: 4: 100%|██████████| 5/5 [01:06<00:00, 13.37s/it]

Iter 100: iter_best colors=24, conflicts=5 | global_best colors=24, conflicts=5
Iter 101: iter_best colors=24, conflicts=5 | global_best colors=24, conflicts=5
  [gc_50_9] Colors: 24, Conflicts: 5

[I 2025-11-29 16:25:24,238] Trial 4 finished with value: 4.0 and parameters: {'iterations': 101, 'num_colors': 35, 'alpha': 2.201281733580727, 'beta': 3.841954371166651, 'rho': 0.24683358443649694, 'ant_count': 31, 'Q': 8.540177486094576}. Best is trial 0 with value: 4.0.

Optimization complete!
Best value: 4.0
Best parameters: {'iterations': 89, 'num_colors': 35, 'alpha': 2.4384921913707727, 'beta': 4.304222254968019, 'rho': 0.24203814411186864, 'ant_count': 28, 'Q': 3.374795050452377}

Generating visualization plots...



/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/code/optuna_tuner.py:338: ExperimentalWarning: optuna.visualization.matplotlib._optimization_history.plot_optimization_history is experimental (supported from v2.2.0). The interface can change in the future.
  fig = optuna.visualization.matplotlib.plot_optimization_history(self.study)
/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/code/optuna_tuner.py:370: ExperimentalWarning: optuna.visualization.matplotlib._param_importances.plot_param_importances is experimental (supported from v2.2.0). The interface can change in the future.
  fig = optuna.visualization.matplotlib.plot_param_importances(self.study)
/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/code/optuna_tuner.py:400: ExperimentalWarning: optuna.visualization.matplotlib._parallel_coordinate.plot_parallel_coordinate is experimental (supported from v2.2.0). The interface can change in the future.
  fig = optuna.visualization.matplotlib.plot_pa

Optimization history saved to: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/figures/aco_study_tiny_dataset_20251129_162417_history.png
Could not plot parameter importances: Tried to import 'sklearn' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'sklearn'.
Parallel coordinate plot saved to: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/figures/aco_study_tiny_dataset_20251129_162417_parallel.png
All plots generated successfully.


Optimization completed!

Best Parameters Found:
Parallel coordinate plot saved to: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/figures/aco_study_tiny_dataset_20251129_162417_parallel.png
All plots generated successfully.


Optimization completed!

Best Parameters Found:


AttributeError: 'Study' object has no attribute 'items'

## 8. Generate Optimization Visualizations

Generate and display plots showing the optimization history, parameter importances, and parallel coordinates.

In [ ]:
# Generate all optimization plots
print("Generating optimization plots...")
tuner.generate_plots(recreate=True)
print(f"Plots saved to: {DATA_ROOT / 'figures'}")

## 9. Test Best Parameters on Testing Dataset

Evaluate the best parameters on the testing dataset.

In [ ]:
# Initialize data loader for testing
test_loader = GraphDataLoader(str(DATA_ROOT), DATASET_NAME)

# Store testing results
testing_results = {}

print("\nEvaluating best parameters on testing dataset...")
print("=" * 70)

# Extract iterations from best_params for ACO runs
best_iterations = best_params.pop('iterations')

# Test on each graph in the testing dataset
for graph_name, graph in test_loader.load_testing_dataset():
    print(f"\nTesting on graph: {graph_name}")
    
    # Create ACO instance with best parameters and double iterations for thorough testing
    test_params = best_params.copy()
    test_params['iterations'] = int(test_params['iterations'] * 2)
    test_params['verbose'] = True
    
    aco = ACOGraphColoring(graph=graph, **test_params)
    
    # Run ACO optimization
    result = aco.run()
    
    # Store results
    testing_results[graph_name] = {
        'color_count': result['color_count'],
        'conflict_count': result['conflict_count'],
        'iterations_used': result['iterations'],
        'best_solution': result['best_solution']
    }
    
    print(f"  Final Colors: {result['color_count']}")
    print(f"  Final Conflicts: {result['conflict_count']}")

print("Testing completed!")
print("\n" + "=" * 70)

In [ ]:
# Export results and generate visualizations
exported_files = export_results(testing_results, best_params, STUDY_NAME, DATA_ROOT)
figure_path = visualize_testing_results(testing_results, STUDY_NAME, DATA_ROOT)

print(f"\nResults exported successfully!")
print(f"Testing figure saved to: {figure_path}")

## 10. Display Results

Show the testing results figure and summary statistics.

In [ ]:
# Display the testing results figure
display(Image(filename=str(figure_path)))

# Print summary statistics
print_summary_statistics(exported_files['summary_df'])

# Print file locations
print_file_locations(STUDY_NAME, DATA_ROOT, exported_files, figure_path)